In [ ]:
# Standard library imports
import sys
import time
import datetime
import itertools
import importlib

# Third-party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import BayesianGaussianMixture
from shapely.geometry import shape
import nimfa
import scipy
from scipy import stats
from pathlib import Path

# Set up plotting style (optional, but professional)
sns.set(style="whitegrid")

# Define the folder path where raw sensor data files are stored

In [ ]:
from pathlib import Path

# Define the path relative to the notebook or project root
FolderPath = Path("../Data/FIN-SWE/")


# List of sensor IDs corresponding to traffic sensors for Finland and Sweden

In [ ]:
Sensors = ["1433", "1432", "1435", "1436", "1431"]

# Loop through each sensor, read its CSV file, and concatenate data into 'all_data'

In [ ]:
# Initialize an empty DataFrame to store combined data from all sensors
all_data = pd.DataFrame()
for s in Sensors:
    # Read CSV file for the current sensor
    sensor_data = pd.read_csv(
        FolderPath + s + '_by_length_minute.csv',
        sep=',',
        usecols=['TMS point id', 'year', 'days', 'hour', 'minute', 'v_type', 'direction', 'total_vehicles', 'date'],
        parse_dates=['date']
    )
    # Concatenate current sensor data to all_data DataFrame
    all_data = pd.concat([all_data, sensor_data])

# Select vehicle type for analysis: options are 'Small', 'Heavy', 'Total', or 'Both'
WhichVehicles = 'Small'

# Create a copy of the combined data for processing
data = all_data.copy()

# Define the date range for filtering the data


In [ ]:
min_date = datetime.datetime(2017, 1, 1)
max_date = datetime.datetime(2023, 12, 31)

# Define a function to aggregate and preprocess the sensor data


In [ ]:
def agg_APIdata_SWEFIN(data, WhichVehicles, min_date=datetime.datetime(2017, 1, 1), max_date=datetime.datetime(2023, 5, 22)):
    """
    Aggregates raw sensor data by vehicle type and time, and formats it for further analysis.

    Parameters:
        data (pd.DataFrame): Raw traffic sensor data.
        WhichVehicles (str): Filter for vehicle types. Options: 'Small', 'Heavy', 'Total', 'Both'.
        min_date (datetime): Minimum datetime filter.
        max_date (datetime): Maximum datetime filter.

    Returns:
        pd.DataFrame: Aggregated and time-indexed vehicle counts.
    """

    # Select relevant columns and reset index
    agg_data = data[['TMS point id', 'hour', 'minute', 'v_type', 'direction', 'total_vehicles', 'date']].reset_index(drop=True).copy()

    # Filter or aggregate data based on vehicle type choice
    if WhichVehicles == 'Total':
        # Sum total vehicles regardless of vehicle type
        agg_data = agg_data.groupby(['TMS point id', 'hour', 'minute', 'direction', 'date'])['total_vehicles'].sum().reset_index()
    elif WhichVehicles == 'Small':
        # Select only small vehicles (length < 5.6m)
        agg_data = agg_data[agg_data['v_type'] == '<5.6m'].drop(columns='v_type').reset_index(drop=True)
    elif WhichVehicles == 'Heavy':
        # Select only heavy vehicles (length >= 5.6m)
        agg_data = agg_data[agg_data['v_type'] == '>=5.6m'].drop(columns='v_type').reset_index(drop=True)

    # Rename columns for clarity
    agg_data = agg_data.rename({'TMS point id': 'sensor_id', 'direction': 'sensor_dir'}, axis=1)

    # Define origin and destination country based on direction
    agg_data['dest_country'] = np.where(agg_data['sensor_dir'] == 2, 'FIN', 'SWE')
    agg_data['origin_country'] = np.where(agg_data['sensor_dir'] == 2, 'SWE', 'FIN')

    # Combine sensor ID and country info for origin and destination
    agg_data["sensor_id"] = agg_data["sensor_id"].astype(str)
    agg_data['sensor_origin'] = agg_data[['sensor_id', 'origin_country']].agg(', '.join, axis=1)
    agg_data['sensor_destination'] = agg_data[['sensor_id', 'dest_country']].agg(', '.join, axis=1)

    # Convert 'hour' and 'minute' columns to timedeltas and add to 'date' to get precise timestamp
    agg_data["minute"] = pd.to_timedelta(agg_data["minute"], unit="min")
    agg_data["hour"] = pd.to_timedelta(agg_data["hour"], unit="h")
    agg_data['date'] = agg_data['date'] + agg_data['hour'] + agg_data['minute']

    # Filter data within the specified date range
    agg_data = agg_data[(agg_data.date > min_date) & (agg_data.date < max_date)].copy().reset_index(drop=True)

    # Update min and max date from filtered data
    min_date = agg_data.date.min()
    max_date = agg_data.date.max()

    # If 'Both' vehicle types requested, return data as is with vehicle type info
    if WhichVehicles == 'Both':
        agg_data = agg_data[['sensor_origin', 'sensor_destination', 'date', 'v_type', 'total_vehicles']].copy()
    else:
        # Otherwise, aggregate total vehicles per sensor origin-destination pair and timestamp
        agg_data = agg_data.groupby(['sensor_origin', 'sensor_destination', 'date'])['total_vehicles'].sum().reset_index()

        # Remove rows with missing vehicle counts (NaNs)
        rm_idx = np.where(np.isnan(agg_data.total_vehicles))
        agg_data = agg_data.drop(index=rm_idx[0])

        # Convert total vehicles to integers
        agg_data['total_vehicles'] = agg_data['total_vehicles'].apply(int)'"'

        # Define a function to reindex data to a complete datetime range with 1-minute frequency, filling missing with zeros
        def fill_missing_timepoints(x):
            return x.reindex(pd.date_range(min_date, max_date, name='date', freq='1min'), fill_value=0)

        # Apply the reindexing function grouped by origin-destination pairs
        agg_data = (
            agg_data.set_index('date')
            .groupby(["sensor_origin", "sensor_destination"])["total_vehicles"]
            .apply(fill_missing_timepoints)
            .reset_index()
        )

        # Pivot the table to have datetime as columns and origin-destination pairs as rows
        agg_data = agg_data.pivot_table(
            index=["sensor_origin", "sensor_destination"],
            columns=["date"],
            values=["total_vehicles"]
        ).droplevel(level=0, axis=1)  # Drop top level of column MultiIndex

    return agg_data

# Run the aggregation function on all_data for 'Small' vehicles and given date range


In [ ]:
agg_data = agg_APIdata_SWEFIN(
    all_data,
    WhichVehicles='Small',
    min_date=datetime.datetime(2017, 1, 1),
    max_date=datetime.datetime(2023, 12, 31)
)

# Fit Gaussian Mixture Models to the aggregated data for the pre-COVID period (2017 to March 1, 2020)

In [ ]:
models_pre, data_pre = utils.fit_period(
    agg_data,
    d1=datetime.date(2017, 1, 1),
    d2=datetime.date(2020, 3, 1),
    hourly=False,
    nSamp=10000,
    Normalize=False,
    N=10,
    seed=1923,
    FitMethod='Bayesian'
)

# Save the fitted models and processed data to disk for later use


In [ ]:
BASE_DIR = Path(".../Data/FinSwe_GMM")  # cleaner & portable
MODELS_FILE = BASE_DIR / "models_fin.pkl"
DATA_FILE = BASE_DIR / "data_fin.pkl"
AGG_DATA_FILE = BASE_DIR / "agg_data_fin.pkl"

# Save
models_pre.to_pickle(MODELS_FILE)
data_pre.to_pickle(DATA_FILE)
agg_data.to_pickle(AGG_DATA_FILE)
